Import clean data:

In [9]:
import os
os.getcwd()


'/Users/jonathankipping/code/syeda-tabassum-rahaman/scam-job-detector/notebooks'

In [10]:
!pwd


/Users/jonathankipping/code/syeda-tabassum-rahaman/scam-job-detector/notebooks


In [11]:
!cd /Users/jonathankipping/code/syeda-tabassum-rahaman/scam-job-detector/

In [12]:
import pandas as pd

df = pd.read_csv('/Users/jonathankipping/code/syeda-tabassum-rahaman/scam-job-detector/raw_data/data_cleaned.csv')
df.head()

,title,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent,department_binary,salary_range_binary,country
0,market intern,food weve create groundbreaking awardwinning c...,food fastgrowing jam beard awardwinning online...,experience content management systems major pl...,miss value,0,1,0,Other,Internship,NaN,NaN,Marketing,0,1,0,US
1,customer service cloud video production,second worlds cloud video production service s...,organise focus vibrant awesomedo passion custo...,expect youyour key responsibility communicate ...,get usthrough part second team gainexperience ...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0,1,0,NZ
2,commission machinery assistant cma,valor service provide workforce solutions meet...,client locate houston actively seek experience...,implement precommissioning commission procedur...,miss value,0,1,0,NaN,NaN,NaN,NaN,NaN,0,0,0,US
3,account executive washington dc,passion improve quality life geography heart e...,company esri – environmental systems research ...,education bachelor ’ master ’ gi business admi...,culture anything corporate—we collaborative cr...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0,1,0,US
4,bill review manager,spotsource solutions llc global human capital ...,job title itemization review managerlocation f...,qualificationsrn license state texasdiploma ba...,full benefit offer,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0,0,0,US


Once you've loaded it up, check what kind of data we're dealing with and take some time to look through some of the examples in the DataFrame.

In [13]:
from scam_job_detector.ML_logic.preprocessor import train_preprocessor, test_preprocessor

In [14]:
df.columns

Index(['title', 'company_profile', 'description', 'requirements', 'benefits',
       'telecommuting', 'has_company_logo', 'has_questions', 'employment_type',
       'required_experience', 'required_education', 'industry', 'function',
       'fraudulent', 'department_binary', 'salary_range_binary', 'country'],
      dtype='object')

In [15]:
df.shape

(17880, 17)

In [16]:
df.tail

<bound method NDFrame.tail of                                                    title  \
0                                          market intern   
1                customer service cloud video production   
2                     commission machinery assistant cma   
3                        account executive washington dc   
4                                    bill review manager   
...                                                  ...   
17875                      account director distribution   
17876                                 payroll accountant   
17877  project cost control staff engineer cost contr...   
17878                                   graphic designer   
17879                         web application developers   

                                         company_profile  \
0      food weve create groundbreaking awardwinning c...   
1      second worlds cloud video production service s...   
2      valor service provide workforce solutions meet...   
3      pa

In [17]:
#Instantiate BERT

from transformers import TFAutoModel

model = TFAutoModel.from_pretrained(
    "prajjwal1/bert-tiny",
    from_pt=True  # load from PyTorch weights
)


TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.decoder.bias', 'bert.embeddings.position_ids', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClas

In [18]:
from transformers import AutoTokenizer

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    "prajjwal1/bert-tiny",
    padding_side="right"  # post-padding
)

# Example sentence
sentence = "My tokenizers and model must match"

# Tokenize and return TF tensors
tokens = tokenizer(
    sentence,
    add_special_tokens=True,   # adds [CLS] and [SEP]
    return_tensors="tf"        # <-- important for TensorFlow models
)

print(tokens)

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.


{'input_ids': <tf.Tensor: shape=(1, 10), dtype=int32, numpy=
array([[  101,  2026, 19204, 17629,  2015,  1998,  2944,  2442,  2674,
          102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 10), dtype=int32, numpy=array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 10), dtype=int32, numpy=array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}


In [19]:
from transformers import TFAutoModel

# Load the Tiny-BERT model
model = TFAutoModel.from_pretrained("prajjwal1/bert-tiny", from_pt=True)

# Example sentence tensors (from previous step)
sentence = "My tokenizers and model must match"
tokens = tokenizer(sentence, add_special_tokens=True, return_tensors="tf")

# Forward pass through the model
outputs = model(**tokens)

# Tokenize and return TF tensors
tokens = tokenizer(sentence, add_special_tokens=True, return_tensors="tf")
input_ids = tokens["input_ids"]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.decoder.bias', 'bert.embeddings.position_ids', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

In [20]:
embeddings = model.predict(input_ids)


1/1 [==============================] - 3s 3s/step


In [21]:
print(type(embeddings))

<class 'transformers.modeling_tf_outputs.TFBaseModelOutputWithPoolingAndCrossAttentions'>


In [22]:
# last_hidden_state: embeddings for each token
token_embeddings = embeddings.last_hidden_state
print("Token embeddings shape:", token_embeddings.shape)

# pooler_output: pooled embedding for [CLS] token
sentence_embedding = embeddings.pooler_output
print("Pooled [CLS] embedding shape:", sentence_embedding.shape)

Token embeddings shape: (1, 10, 128)
Pooled [CLS] embedding shape: (1, 128)


In [23]:
outputs = model(**tokens)
last_hidden_state = outputs.last_hidden_state
print(last_hidden_state.shape)

(1, 10, 128)


FINE TUNED MODEL


In [24]:
from transformers import TFAutoModelForSequenceClassification

# Load Tiny-BERT for binary classification (num_labels=2)
model_cls = TFAutoModelForSequenceClassification.from_pretrained(
    "prajjwal1/bert-tiny",
    num_labels=2,
    from_pt=True  # load PyTorch weights if needed
)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
model_cls.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  4385920   
                                                                 
 dropout_21 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  258       
                                                                 
Total params: 4386178 (16.73 MB)
Trainable params: 4386178 (16.73 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
import numpy as np
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from sklearn.model_selection import train_test_split

# -----------------------------
# Data Targets
# -----------------------------

X = df.drop(columns=["fraudulent"])
y = df["fraudulent"]

# -----------------------------
# Combine text columns
# -----------------------------
text_columns = [
    "title",
    "company_profile",
    "description",
    "requirements",
    "benefits"
]

combined_text = X[text_columns].fillna("").agg(" ".join, axis=1).to_list()

# -----------------------------
# Train-test split
# -----------------------------

X_train, X_test, train_mask, test_mask, y_train, y_test = train_test_split(
    combined_text, y,
    attention_mask,
    test_size=0.2,
    random_state=42,
    stratify=y
)


# -----------------------------
# Tokenizer
# -----------------------------
tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-tiny")

train_tokens = tokenizer(
    X_train,
    padding="max_length",
    truncation=True,
    max_length=256,
    return_tensors="np",
)

test_tokens = tokenizer(
    X_test,
    padding="max_length",
    truncation=True,
    max_length=256,
    return_tensors="np",
)

# -----------------------------
# Load BERT Tiny model
# -----------------------------
model = TFAutoModelForSequenceClassification.from_pretrained(
    "prajjwal1/bert-tiny",
    num_labels=2,
    from_pt=True
)

# -----------------------------
# Compile Model
# -----------------------------
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

# -----------------------------
# Fit Model
# -----------------------------
history = model.fit(
    {
        "input_ids": train_tokens["input_ids"],
        "attention_mask": train_tokens["attention_mask"],
    },
    y_train.values,
    validation_split=0.1,
    epochs=3,
    batch_size=16
)

# -----------------------------
# Evaluate
# -----------------------------
test_loss, test_acc = model.evaluate(
    {
        "input_ids": test_tokens["input_ids"],
        "attention_mask": test_tokens["attention_mask"],
    },
    y_test.values
)

print("Test Accuracy:", test_acc)

# Predict on test set
pred_logits = model.predict(
    {"input_ids": y_test, "attention_mask": test_mask}
).logits

# Convert logits → class labels
y_pred = np.argmax(pred_logits, axis=1)

# Print metrics
print(f"""
Model Performance
Recall:             {recall_score(y_test, y_pred):.4f}
Precision:          {precision_score(y_test, y_pred):.4f}
Balanced Accuracy:  {balanced_accuracy_score(y_test, y_pred):.4f}
F1 Score:           {f1_score(y_test, y_pred):.4f}
""")


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
190/805 [======>.......................] - ETA: 1:32 - loss: 0.2935 - accuracy: 0.9454

KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import recall_score, precision_score, balanced_accuracy_score, f1_score

In [ ]:
import numpy as np
from sklearn.metrics import recall_score, precision_score, balanced_accuracy_score, f1_score

# Predict on test set
pred_logits = model.predict(
    {"input_ids": test_ids, "attention_mask": test_mask}
).logits

# Convert logits → class labels
y_pred = np.argmax(pred_logits, axis=1)

# Print metrics
print(f"""
Model Performance
Recall:             {recall_score(y_test, y_pred):.4f}
Precision:          {precision_score(y_test, y_pred):.4f}
Balanced Accuracy:  {balanced_accuracy_score(y_test, y_pred):.4f}
F1 Score:           {f1_score(y_test, y_pred):.4f}
""")


112/112 [==============================] - 11s 95ms/step

Model Performance
Recall:             0.6012
Precision:          0.8387
Balanced Accuracy:  0.7976
F1 Score:           0.7003



In [ ]:
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer
from sklearn.model_selection import train_test_split

# ----------------------------
# Load tokenizer
# ----------------------------
tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-small", padding_side="right")

# ----------------------------
# Prepare text + labels
# ----------------------------
texts = df["description"].fillna("").tolist()
labels = df["fraudulent"].astype(int).values  # 0 = legit, 1 = fraudulent

# ----------------------------
# Tokenize
# ----------------------------
tokenized = tokenizer(
    texts,
    max_length=256,
    padding="max_length",
    truncation=True,
    return_tensors="np"   # better for train_test_split
)

input_ids = tokenized["input_ids"]
attention_mask = tokenized["attention_mask"]

# ----------------------------
# Train-test split
# ----------------------------
train_ids, test_ids, train_mask, test_mask, y_train, y_test = train_test_split(
    input_ids,
    attention_mask,
    labels,
    test_size=0.2,
    random_state=42,
    stratify=labels
)


# ----------------------------
# Load model
# ----------------------------
model = TFAutoModelForSequenceClassification.from_pretrained(
    "prajjwal1/bert-small",
    num_labels=2,
    from_pt=True
)

# ----------------------------
# Compile
# ----------------------------
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

# ----------------------------
# Fit model
# ----------------------------
history = model.fit(
    {
        "input_ids": train_ids,
        "attention_mask": train_mask
    },
    y_train,
    validation_split=0.1,
    epochs=3,
    batch_size=16
)

# ----------------------------
# Evaluate
# ----------------------------
test_loss, test_acc = model.evaluate(
    {
        "input_ids": test_ids,
        "attention_mask": test_mask
    },
    y_test
)

print("Test accuracy:", test_acc)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
615/805 [=====================>........] - ETA: 6:13 - loss: 0.1469 - accuracy: 0.9591

KeyboardInterrupt: 